# Audio 
Be careful not to load unless it is requested

In [2]:
%reload_ext autoreload
%autoreload 2
import colabexts
from colabexts.jcommon import *

import os, sys, datetime, re, json, importlib
from collections import defaultdict
from sys import modules
from IPython.display import HTML, Javascript
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

%matplotlib inline
pd.set_option('display.max_colwidth', 40)
pd.set_option('display.max_rows', 16)

# Trasncribe using Whisper

In [1]:
%%writefile ../geoapp/transcribe.py
#!/usr/bin/env python

# DO NOT EDIT THIS FILE : Generated from scribe/notebooks/transcribe.ipynb
#!pip install git+https://github.com/openai/whisper.git

from mangorest import mango
from  mangorest.mango import webapi
import colabexts.utils as colabexts_utils
import io, os, librosa, logging, sys

logger = logging.getLogger( "geoapp" )

# ----------------------------------------------------------------------
fn="/Users/snarayan/Desktop/data/audio/test.wav"
# ------------------------------------------------------------------------------
whisperModel  = None
def getModel():
    global whisperModel
    if ( whisperModel is None):
        import whisper
        whisperModel = whisper.load_model("base")
        #whisperModel.DecodingOptions(fp16 = True) 
    return whisperModel

@webapi("/geoaudio/transcribe/")
def transcribe(request=None, file="/tmp/test.wav", offset=0, duration=None, save="", **kwargs): 
    if ( request and request.FILES.getlist('file')):
        for f in request.FILES.getlist('file'):
            file = f.read()
            if ( save ):
                filename = save or '/tmp/test.wav'
                logger.info(f'GOT FILE will save to {filename} and transcribe')
                with open (f"{filename}", "wb") as ff:
                    ff.write(file)
            break;

    if (type(file) == str):
        logger.info(f'File ...{file}')
        if ( not file or not os.path.exists(file)):
            return f"{file}"
        data, sample_rate = librosa.load(file, offset=offset, duration=duration, mono=True, sr=16000)
    else:
        content = io.BytesIO(file)
        logger.info(f'Bytes ... {content.getbuffer().nbytes}')
        data, sample_rate = librosa.load(content,sr=16000, mono=True, offset=offset, duration=duration)
    
    audio, _ = librosa.effects.trim(data)

    out = getModel().transcribe(data)
    #print(out)
    return out['text']

if __name__ == '__main__' and not colabexts_utils.inJupyter():
    transcribe(sys.argv[1])

Overwriting ../geoapp/transcribe.py


In [9]:
transcribe(file="/tmp/test.wav")

2024-07-10 08:40:32,804 geoapp INFO: File .../tmp/test.wav
/Users/snarayan/venv/py311/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


" Hello, hello, hello, yeah, what's up? What's going on?"

In [42]:
c = open("/tmp/test.wav", "rb").read()
b=io.BytesIO(c)

In [46]:
len(c), b.getbuffer().nbytes

(245804, 245804)

In [48]:
data, sample_rate = librosa.load(b,sr=16000, mono=True, offset=0)


In [50]:
len(data)
model.transcribe(data)

{'text': " What do you think? Is this going to work? I think I'm trying to make it a longer recording just to make sure that everything works.",
 'segments': [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 4.88,
   'text': " What do you think? Is this going to work? I think I'm trying to make it a longer",
   'tokens': [50364,
    708,
    360,
    291,
    519,
    30,
    1119,
    341,
    516,
    281,
    589,
    30,
    286,
    519,
    286,
    478,
    1382,
    281,
    652,
    309,
    257,
    2854,
    50608],
   'temperature': 0.0,
   'avg_logprob': -0.3237735203334263,
   'compression_ratio': 1.297029702970297,
   'no_speech_prob': 0.13521838188171387},
  {'id': 1,
   'seek': 0,
   'start': 4.88,
   'end': 9.120000000000001,
   'text': ' recording just to make sure that everything works.',
   'tokens': [50608, 6613, 445, 281, 652, 988, 300, 1203, 1985, 13, 50820],
   'temperature': 0.0,
   'avg_logprob': -0.3237735203334263,
   'compression_ratio': 1.29702970297029